In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
project_folder_path = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database'
sys.path.append(project_folder_path)
from utils.db_functs import *
from utils.requests_functs import *
from utils.html_functs import *

/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database


/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Database and directories

In [2]:
db_file_name = 'cev-database-coordinates-v1.db'
db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', db_file_name)

## 1) Define parameters to run a `search`
This can be a new search or retake a previous one

In [3]:
#search_id = '0f62f13c-2f06-43f9-b10c-323673190d29'
search_id = '010576e8-90f1-42c4-88bc-3965b5c4c238'
#search_date = '2024-05-28'
search_date = '2024-04-21'

## 2) Define a dataframe with download coordinates
The dataframe must contain 1 row per `comuna` meaning 348 rows in total.

In [4]:
query = f"""
        SELECT * FROM evals_html_download_coordinates
        WHERE search_id = "{search_id}";
        """
print(query)


        SELECT * FROM evals_html_download_coordinates
        WHERE search_id = "010576e8-90f1-42c4-88bc-3965b5c4c238";
        


In [5]:
df = create_dataframe_from_query(db_file_path, query)
# Drop specified columns
columns_to_drop = ['id']
df = df.drop(columns=columns_to_drop)
df.head()

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eventtarget,viewstate,eventargument,total_evals,total_pages
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$1,581,59
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$2,581,59
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$3,581,59
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$4,581,59
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$5,581,59


Set additional features to track the status of downloads as well as times and file names

In [7]:
df['status'] = None
df['html_filename'] = None
df['downloaded_at'] = None
df.head()

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eventtarget,viewstate,eventargument,total_evals,total_pages,status,html_filename,downloaded_at
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$1,581,59,None,None,None
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$2,581,59,None,None,None
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$3,581,59,None,None,None
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$4,581,59,None,None,None
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$5,581,59,None,None,None


## 3) Check first if any html files has been downloaded
This is useful in case a seach has been run and has not completed

### 3.1) Read all html files for the corresponding `search_date` and `search_id`

In [8]:
# Define the directory path
html_files_dir = os.path.join(project_folder_path, 'data', 'raw', f'{search_date}_{search_id}', '2_evals_comuna_page', 'html_files')

# Check if the directory exists
if not os.path.exists(html_files_dir):
    # If it doesn't exist, create the directory
    os.makedirs(html_files_dir)
    print(f"Directory '{html_files_dir}' created.")

# List all files in the directory
html_file_paths = find_html_files(html_files_dir)
print(f'The directory: {html_files_dir} contains {len(html_file_paths)} out of {df.shape[0]}')

The directory: /mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database/data/raw/2024-04-21_010576e8-90f1-42c4-88bc-3965b5c4c238/2_evals_comuna_page/html_files contains 7300 out of 16014


### 3.2) Fill on dataframe all those rows corresponding to files already downloaded

In [9]:
for html_file_path in html_file_paths:
    html_file_name = os.path.split(html_file_path)[-1]
    html_file_list = html_file_name.split('_')
    region_id = int(html_file_list[0])
    comuna_id = int(html_file_list[1])
    tipo_evaluacion = int(html_file_list[2])
    eventargument = '$'.join(html_file_list[3].split('-')[:2])
    number_of_pages = str(html_file_list[3].split('-')[-1])
    date = html_file_list[4]
    time = html_file_list[5].split('.')[0].replace('-', ':')
    date_time = date + ' ' + time
    row = df[df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str)+ '_' + df['eventargument'].astype(str) == str(region_id) + '_' + str(comuna_id) + '_' + str(tipo_evaluacion) + '_' + str(eventargument)]
    if not row.empty:
        df.loc[row.index, 'status'] = 'Successful'
        df.loc[row.index, 'html_filename'] = html_file_name
        df.loc[row.index, 'downloaded_at'] = date_time    
df.head() 

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eventtarget,viewstate,eventargument,total_evals,total_pages,status,html_filename,downloaded_at
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$1,581,59,Successful,1_5_1_Page-1-59_2024-04-22_08-49-23.html,2024-04-22 08:49:23
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$2,581,59,Successful,1_5_1_Page-2-59_2024-04-22_08-49-26.html,2024-04-22 08:49:26
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$3,581,59,Successful,1_5_1_Page-3-59_2024-04-22_08-49-30.html,2024-04-22 08:49:30
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$4,581,59,Successful,1_5_1_Page-4-59_2024-04-22_08-49-34.html,2024-04-22 08:49:34
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ctl00$ContentPlaceHolder1$grdViviendasPre,CX7AKjonPjA6VlUD1xG2Pw+JH1Nl+NjtHGau9bKhUXmzo6...,Page$5,581,59,Successful,1_5_1_Page-5-59_2024-04-22_08-49-37.html,2024-04-22 08:49:37


### Download html files: 1 file per commune

In [10]:
HOME_URL = 'http://calificacionenergeticaweb.minvu.cl/Publico/BusquedaVivienda.aspx'

MAX_ATTEMPTS = 5
DELAY_BETWEEN_ATTEMPTS = 15

attempts = 0
while attempts < MAX_ATTEMPTS:
    all_successful = True  # Flag to track if all rows are successful in this attempt
    communes_to_process = df[df['status'] != 'Successful'].shape[0]
    print(f"Attempt {attempts + 1} of {MAX_ATTEMPTS}: {communes_to_process} communes to process")

    for index, row in df.iterrows():
        if df.loc[index, 'status'] != 'Successful':
            region_id = str(row['region_id'])
            commune_id = str(row['comuna_id'])
            tipo_evaluacion = str(row['tipo_evaluacion'])
            eventtarget = row['eventtarget']
            eventargument = row['eventargument']
            number_of_pages = str(row['total_pages'])
            viewstate = row['viewstate']
            form_data = form_data_evaluacion(eventtarget, eventargument, region_id, commune_id, tipo_evaluacion, viewstate)
            
            print(f" * Processing {region_id}, {commune_id}")
           
            try:
                response = requests.post(HOME_URL, data=form_data)

                if response.status_code == 200:
                    # Record the current time after making the request
                    request_time = datetime.now()
                    downloaded_on = request_time.strftime("%Y-%m-%d")
                    downloaded_at = request_time.strftime("%Y-%m-%d %H:%M:%S")
                    
                    df.loc[index, 'status'] = 'Successful'
                    
                    # Saving html data into a local folder
                    html_path = os.path.join(html_files_dir, region_id)
                   
                    if not os.path.exists(html_path):
                        os.makedirs(html_path)
                        
                    html_filename = region_id + "_" + commune_id + "_" + tipo_evaluacion + "_" + eventargument.split('$')[0] + '-' + eventargument.split('$')[-1] + '-' + str(number_of_pages) + '_' + request_time.strftime("%Y-%m-%d_%H-%M-%S") + '.html'

                    

                    with open(os.path.join(html_path, html_filename), "wb") as f:
                        f.write(response.content)                    
                    
                    df.loc[index, 'html_filename'] = downloaded_at
                    df.loc[index, 'downloaded_at'] = downloaded_at
                else:
                    #df.loc[index, 'status'] = 'Failed'
                    print(f"Request failed for region {region_id}, commune {commune_id}. Skipping to the next row.")
                    continue  # Skip to the next iteration of the loop
                #df.loc[index, 'status_code'] = response.status_code
            
            except Exception as e:
                print(f"An error occurred for region {region_id}, commune {commune_id}: {str(e)}")
                #df.loc[index, 'status'] = 'Failed'
                continue  # Skip to the next iteration of the loop
            
    
    all_successful = all_successful and (df['status'] == 'Successful').all()

    if all_successful:
        break  # Exit the loop if all rows are successful
    else:
        attempts += 1
        time.sleep(DELAY_BETWEEN_ATTEMPTS)  # Delay between attempts


In [11]:
# Drop specified columns
columns_to_drop = ['eventtarget', 'eventargument', 'viewstate', 'total_evals']
out_df = df.drop(columns=columns_to_drop).dropna().reset_index(drop=True)
out_df

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,total_pages,status,html_filename,downloaded_at
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,59,Successful,1_5_1_Page-1-59_2024-04-22_08-49-23.html,2024-04-22 08:49:23
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,59,Successful,1_5_1_Page-2-59_2024-04-22_08-49-26.html,2024-04-22 08:49:26
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,59,Successful,1_5_1_Page-3-59_2024-04-22_08-49-30.html,2024-04-22 08:49:30
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,59,Successful,1_5_1_Page-4-59_2024-04-22_08-49-34.html,2024-04-22 08:49:34
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,59,Successful,1_5_1_Page-5-59_2024-04-22_08-49-37.html,2024-04-22 08:49:37
...,...,...,...,...,...,...,...,...,...,...
7295,5,1,2,6,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,60,Successful,1_5_2_Page-6-60_2024-05-29_21-23-33.html,2024-05-29 21:23:33
7296,5,1,2,7,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,60,Successful,1_5_2_Page-7-60_2024-05-29_21-23-38.html,2024-05-29 21:23:38
7297,5,1,2,8,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,60,Successful,1_5_2_Page-8-60_2024-05-29_21-23-43.html,2024-05-29 21:23:43
7298,5,1,2,9,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,60,Successful,1_5_2_Page-9-60_2024-05-29_21-23-48.html,2024-05-29 21:23:48


In [12]:
insert_unique_rows_from_dataframe(db_file_path, 'evals_html_downloaded_files', out_df, unique_columns=['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id'])

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (5, 1, 1, 1, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (5, 1, 1, 2, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (5, 1, 1, 3, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (5, 1, 1, 4, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (5, 1, 1, 5, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (9, 2, 1, 53, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (9, 2, 1, 54, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (9, 2, 1, 55, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (9, 2, 1, 56, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (9, 2, 1, 57, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluaci

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (314, 2, 1, 8, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (314, 2, 1, 9, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (314, 2, 1, 10, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (314, 2, 1, 11, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (314, 2, 1, 12, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (68, 4, 1, 93, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (68, 4, 1, 94, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (68, 4, 1, 95, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (68, 4, 1, 96, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (68, 4, 1, 97, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_eva

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (126, 4, 1, 147, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (126, 4, 1, 148, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (126, 4, 1, 149, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (126, 4, 1, 150, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (126, 4, 1, 151, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id',

...

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (194, 13, 1, 66, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (194, 13, 1, 67, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (194, 13, 1, 68, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (194, 13, 1, 69, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (194, 13, 1, 70, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id',

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (229, 13, 1, 24, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (229, 13, 1, 25, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (229, 13, 1, 26, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (229, 13, 1, 27, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (229, 13, 1, 28, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id',

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (233, 13, 1, 110, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (233, 13, 1, 111, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (233, 13, 1, 112, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (233, 13, 1, 113, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (233, 13, 1, 114, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (281, 13, 1, 6, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (281, 13, 1, 7, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (281, 13, 1, 8, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (281, 13, 1, 9, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (281, 13, 1, 10, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'ti

Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (294, 13, 1, 19, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (294, 13, 1, 20, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (294, 13, 1, 21, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (294, 13, 1, 22, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id']: (294, 13, 1, 23, '010576e8-90f1-42c4-88bc-3965b5c4c238'). Skipping insertion.
Duplicate row found for unique columns ['comuna_id', 'region_id',